In [65]:
import pandas as pd
import numpy as np

In [66]:
def change_features(df):
    df['AVG_returns'] = (df['AVG_returns'] + 1) ** 252 - 1
    df['Risk'] = (df['Risk'] + 1) ** np.sqrt(252) - 1
    df['Alpha'] = df['Alpha'].round(4)
    df['Drawdown'] = -df['Drawdown']
    return df 

In [67]:
dict_category2emb = {
    'topological features' : [
        'PersImage_2',
        'PersLandscape_1',
        'PersImage_1',
        'PersLandscape_2',
        'topo_features'                 
      ],
    
    'Dimensionality reduction' : ['umap', 'pca', 'fastica', 'tnse'],
    
    'transformers' : [ 'transformer_embds1',
 'transformer_embds2',
 'transformer_embds3',
 'transformer_embds_sum_23',
 'transformer_embds_sum_all',],
    
    'autoencoders' : [ 'autoencoder_conv',
 'autoencoder_lstm',
 'autoencoder_mlp',],
    
    #'baselines' : ['sp500', 'sectors'],
    'index': ['sp500'],
    'ecomonic sectors': ['sectors'],
    
    'classical features' : [ 'tsfresh', 'table_finance_features'],
    
#     'time_clustering' : ['KShape',
#  'MSTcorr',
#  'TimeSeriesKMeans'],
    
    'time series embedding' : ['ts2vec', 'signal2vec_embds']
}

dict_emb2cat = {method:categ for categ, list_methods in dict_category2emb.items() for method in list_methods }

# Data loading

In [68]:
fin_results_2012 = pd.read_csv('results/usa_2012/financial_metric_False.csv', index_col=0)
fin_results_2015 = pd.read_csv('results/usa_2015/financial_metric_False.csv', index_col=0)
fin_results_2018 = pd.read_csv('results/usa_2018/financial_metric_False.csv', index_col=0)

clust_results_2012 = pd.read_csv('results/usa_2012/clust_metric_False.csv', index_col=0)
clust_results_2015 = pd.read_csv('results/usa_2012/clust_metric_False.csv', index_col=0)
clust_results_2018 = pd.read_csv('results/usa_2012/clust_metric_False.csv', index_col=0)
fin_results_2012.head()

,AVG_returns,Risk,Beta,Alpha,Sharpe,VaR,Drawdown,Recovery,clust_model,emb_model
port,0.000517,0.016028,0.329642,0.000345,0.024955,-0.026447,-37.243770,385.0,KMeans,ts2vec
port,0.001062,0.014736,0.384230,0.000881,0.064101,-0.024314,-27.391429,259.0,Agg,ts2vec
port,0.000656,0.007743,0.752471,0.000414,0.069636,-0.012776,-11.716343,185.0,KMeans,PersImage_2
port,0.000659,0.008137,0.736411,0.000420,0.066596,-0.013426,-11.172857,183.0,Agg,PersImage_2
port,0.000633,0.015816,0.339472,0.000459,0.032596,-0.026096,-38.037391,385.0,KMeans,PersLandscape_1


In [69]:
def load_fin_clust(exp_name):
    dict_data = {
        "finance":pd.read_csv(f'results/{exp_name}/financial_metric_False.csv', index_col=0),
        "clustering":pd.read_csv(f'results/{exp_name}/clust_metric_False.csv', index_col=0)
    }
    return dict_data


In [70]:
experiments = ['usa_2012', 'usa_2015', 'usa_2018']
dict_results = {exp:load_fin_clust(exp) for exp in experiments}

In [71]:
def build_entire_table(fin_results, clust_results, dict_emb2cat, sort_metric='Risk'):
    fin_results = change_features(fin_results)

    df_to_rebuild = fin_results
    df_to_rebuild = pd.merge(clust_results.drop(columns=['clust_model']), df_to_rebuild, on='emb_model', how='outer')

    df_best_clustering = df_to_rebuild.sort_values([sort_metric], ascending=False)
    df_best_clustering = df_best_clustering.groupby('emb_model', as_index=False, sort=True).first()

    df_best_clustering['category'] = df_best_clustering['emb_model'].map(dict_emb2cat)
    return df_best_clustering.sort_values(sort_metric)

In [72]:
dict_results_selected = {exp:build_entire_table(
        dict_results[exp]['finance'], 
        dict_results[exp]['clustering'], 
        dict_emb2cat
        ) 
    for exp in experiments
}

# all experiments analysis

In [73]:
all_experiments = pd.DataFrame([])

for exp_name, exp_data in dict_results_selected.items():
    exp_data['exp_name'] = exp_name
    
    all_experiments = pd.concat([all_experiments, exp_data])
    
print(all_experiments.shape)
all_experiments.head()

(63, 16)


,emb_model,DB,HC,Sil,hom,AVG_returns,Risk,Beta,Alpha,Sharpe,VaR,Drawdown,Recovery,clust_model,category,exp_name
1,PersImage_2,1.724300,61.553752,0.145029,0.099462,0.180643,0.137287,0.736411,0.0004,0.066596,-0.013426,11.172857,183.0,Agg,topological features,usa_2012
10,sp500,NaN,NaN,NaN,NaN,0.073929,0.142009,1.000000,0.0000,0.019734,-0.013860,14.183248,253.0,sp500,index,usa_2012
0,PersImage_1,2.113002,38.591573,0.112135,0.085452,0.230194,0.143126,0.790175,0.0006,0.083330,-0.013962,11.145127,128.0,Agg,topological features,usa_2012
11,table_finance_features,1.243502,76.237093,0.159563,0.134707,0.293567,0.185312,0.500845,0.0008,0.084022,-0.017765,14.913612,184.0,KMeans,classical features,usa_2012
9,sectors,4.019210,7.379893,-0.029870,1.000000,0.304948,0.204896,0.517192,0.0009,0.079542,-0.019488,25.386463,256.0,sectors,ecomonic sectors,usa_2012


In [74]:
features_to_drop = ['DB', 'HC', 'Sil', 'hom', 'Beta', 'Alpha', 
                    'AVG_returns', 'VaR', 'Drawdown', 'Recovery']
all_experiments = all_experiments.drop(features_to_drop, axis=1)
all_experiments = all_experiments

all_experiments.head()

,emb_model,Risk,Sharpe,clust_model,category,exp_name
1,PersImage_2,0.137287,0.066596,Agg,topological features,usa_2012
10,sp500,0.142009,0.019734,sp500,index,usa_2012
0,PersImage_1,0.143126,0.083330,Agg,topological features,usa_2012
11,table_finance_features,0.185312,0.084022,KMeans,classical features,usa_2012
9,sectors,0.204896,0.079542,sectors,ecomonic sectors,usa_2012


In [75]:
all_experiments[all_experiments['category'] == 'topological features']

,emb_model,Risk,Sharpe,clust_model,category,exp_name
1,PersImage_2,0.137287,0.066596,Agg,topological features,usa_2012
0,PersImage_1,0.143126,0.083330,Agg,topological features,usa_2012
3,PersLandscape_2,0.281690,0.041332,Agg,topological features,usa_2012
2,PersLandscape_1,0.282873,0.032596,KMeans,topological features,usa_2012
12,topo_features,0.301772,0.041673,Agg,topological features,usa_2012
1,PersImage_2,0.142336,0.128478,Agg,topological features,usa_2015
0,PersImage_1,0.147628,0.059114,KMeans,topological features,usa_2015
12,topo_features,0.179310,0.099889,Agg,topological features,usa_2015
3,PersLandscape_2,0.218190,0.150288,KMeans,topological features,usa_2015
2,PersLandscape_1,0.278914,0.125858,Agg,topological features,usa_2015


In [76]:
all_exp_melt = pd.melt(all_experiments, id_vars=['exp_name', 'category', 'emb_model', 'clust_model'], 
        value_vars=['Risk', 'Sharpe'], var_name='metric', value_name='value')

all_exp_pivot = all_exp_melt.pivot(index='emb_model', columns=['exp_name', 'metric'], values=['value'])
all_exp_pivot.columns = all_exp_pivot.columns.droplevel()
all_exp_pivot = all_exp_pivot.sort_index(axis=1, level=0)
all_exp_pivot.sort_values([('usa_2018', 'Risk')], inplace=True)

all_exp_pivot

exp_name                usa_2012            usa_2015            usa_2018  \
metric                      Risk    Sharpe      Risk    Sharpe      Risk   
emb_model                                                                  
sp500                   0.142009  0.019734  0.136343  0.043638  0.289158   
PersImage_2             0.137287  0.066596  0.142336  0.128478  0.335643   
PersImage_1             0.143126  0.083330  0.147628  0.059114  0.337164   
fastica                 0.279785  0.044366  0.225570  0.078007  0.361338   
topo_features           0.301772  0.041673  0.179310  0.099889  0.367651   
pca                     0.279785  0.044366  0.225570  0.078007  0.382380   
ts2vec                  0.287141  0.024955  0.290068  0.098511  0.390234   
transformer_embds2      0.211621  0.076999  0.169785  0.147098  0.390733   
table_finance_features  0.185312  0.084022  0.176772  0.132535  0.392020   
transformer_embds5      0.211621  0.076999  0.160075  0.158944  0.393402   
autoencoder_lstm        0.237643  0.065529  0.219016  0.121365  0.399237   
tsfresh                 0.267469  0.018736  0.243374  0.113466  0.403770   
transformer_embds3      0.214303  0.081541  0.169452  0.148898  0.407732   
PersLandscape_2         0.281690  0.041332  0.218190  0.150288  0.410371   
sectors                 0.204896  0.079542  0.166120  0.135522  0.415533   
transformer_embds4      0.236646  0.042340  0.254810  0.133101  0.419380   
transformer_embds1      0.280571  0.046580  0.236600  0.122942  0.419430   
umap                    0.220492  0.075281  0.192902  0.110317  0.425462   
PersLandscape_1         0.282873  0.032596  0.278914  0.125858  0.426972   
autoencoder_conv        0.316566  0.043027  0.205836  0.143847  0.441784   
autoencoder_mlp         0.297101  0.040065  0.240412  0.113466  0.450260   

exp_name                          
metric                    Sharpe  
emb_model                         
sp500                   0.014858  
PersImage_2             0.063872  
PersImage_1             0.050032  
fastica                 0.037374  
topo_features           0.049890  
pca                     0.025618  
ts2vec                  0.072554  
transformer_embds2      0.068911  
table_finance_features  0.051444  
transformer_embds5      0.066072  
autoencoder_lstm        0.067996  
tsfresh                 0.061094  
transformer_embds3      0.059415  
PersLandscape_2         0.066810  
sectors                 0.042644  
transformer_embds4      0.022160  
transformer_embds1      0.027812  
umap                    0.034040  
PersLandscape_1         0.068696  
autoencoder_conv        0.041836  
autoencoder_mlp         0.055189

## category results

In [81]:
all_experiments = all_experiments.sort_values(['Risk'], ascending=False)
all_experiments_gr = all_experiments.groupby(['exp_name', 'category'], as_index=False, sort=True).first()

all_exp_melt = pd.melt(all_experiments_gr, id_vars=['exp_name', 'category', 'emb_model', 'clust_model'], 
        value_vars=['Risk', 'Sharpe'], var_name='metric', value_name='value')

all_exp_pivot = all_exp_melt.pivot(index='category', columns=['exp_name', 'metric'], values=['value'])
all_exp_pivot.columns = all_exp_pivot.columns.droplevel()
all_exp_pivot = all_exp_pivot.sort_index(axis=1, level=0)
all_exp_pivot.sort_values([('usa_2018', 'Risk')], inplace=True)

all_exp_pivot.to_excel('results/tables/categories.xlsx')

all_exp_pivot


exp_name                  usa_2012            usa_2015            usa_2018  \
metric                        Risk    Sharpe      Risk    Sharpe      Risk   
category                                                                     
index                     0.142009  0.019734  0.136343  0.043638  0.289158   
time series embedding     0.287141  0.024955  0.290068  0.098511  0.390234   
classical features        0.267469  0.018736  0.243374  0.113466  0.403770   
ecomonic sectors          0.204896  0.079542  0.166120  0.135522  0.415533   
transformers              0.280571  0.046580  0.236600  0.122942  0.419430   
Dimensionality reduction  0.279785  0.044366  0.225570  0.078007  0.425462   
topological features      0.301772  0.041673  0.278914  0.125858  0.426972   
autoencoders              0.316566  0.043027  0.240412  0.113466  0.450260   

exp_name                            
metric                      Sharpe  
category                            
index                     0.014858  
time series embedding     0.072554  
classical features        0.061094  
ecomonic sectors          0.042644  
transformers              0.027812  
Dimensionality reduction  0.034040  
topological features      0.068696  
autoencoders              0.055189

## only topological

In [80]:
all_experiments = all_experiments.sort_values(['Risk'], ascending=False)
save_categories = ['topological features', 'index', 'ecomonic sectors']
all_experiments_selected = all_experiments.query('category in @save_categories')
#all_experiments_gr = all_experiments.groupby(['exp_name', 'category'], as_index=False, sort=True).first()

all_exp_melt = pd.melt(all_experiments_selected, id_vars=['exp_name', 'category', 'emb_model', 'clust_model'], 
        value_vars=['Risk', 'Sharpe'], var_name='metric', value_name='value')

all_exp_pivot = all_exp_melt.pivot(index='emb_model', columns=['exp_name', 'metric'], values=['value'])
all_exp_pivot.columns = all_exp_pivot.columns.droplevel()
all_exp_pivot = all_exp_pivot.sort_index(axis=1, level=0)
all_exp_pivot.sort_values([('usa_2018', 'Risk')], inplace=True)
dict_rename = {'sp500':'index', 'sectors': 'ecomonic sectors '}
all_exp_pivot.index = [dict_rename[cat_name] if cat_name in dict_rename else cat_name
                       for cat_name in all_exp_pivot.index]

all_exp_pivot.to_excel('results/tables/topological.xlsx')

all_exp_pivot

exp_name           usa_2012            usa_2015            usa_2018          
metric                 Risk    Sharpe      Risk    Sharpe      Risk    Sharpe
index              0.142009  0.019734  0.136343  0.043638  0.289158  0.014858
PersImage_2        0.137287  0.066596  0.142336  0.128478  0.335643  0.063872
PersImage_1        0.143126  0.083330  0.147628  0.059114  0.337164  0.050032
topo_features      0.301772  0.041673  0.179310  0.099889  0.367651  0.049890
PersLandscape_2    0.281690  0.041332  0.218190  0.150288  0.410371  0.066810
ecomonic sectors   0.204896  0.079542  0.166120  0.135522  0.415533  0.042644
PersLandscape_1    0.282873  0.032596  0.278914  0.125858  0.426972  0.068696